### Week 5 Day 4

AutoGen Core - Distributed



In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some key reasons in favor of choosing AutoGen for your new AI Agent project:

1. **Human-in-the-Loop Capabilities**: AutoGen supports workflows that allow human oversight, enabling you to guide or approve agent decisions at crucial moments. This adds a layer of control and reliability to your applications.

2. **Context Management**: AutoGen efficiently handles context management, allowing agents to maintain coherent communication without needing manual intervention. This streamlines interactions and improves overall efficiency.

3. **Scalability**: The architecture of AutoGen is designed to scale effectively, making it easier to expand the functionality of your AI agents as project requirements evolve.

4. **Inter-Agent Communication**: AutoGen simplifies communication between multiple agents, ensuring they can share information seamlessly and work collaboratively towards common goals.

5. **Reduced Development Overhead**: By automating many aspects of agent orchestration and communication, AutoGen can reduce the development time and complexity associated with building and managing AI agents.

6. **Adaptability**: Its flexible framework allows for easy adjustments and integrations with other systems or technologies, enabling customization as per your project's specific needs.

Using AutoGen can streamline the development of AI agents, improve their performance, and enhance user control over agent behaviors. 

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **High Cost Structure**: AutoGen uses a user-based pricing model, which can become quite costly as the number of users or team members increases.

2. **Import Function Issues**: There can be problems with importing files, particularly PDFs, leading to formatting issues. This can make the initial setup and content integration challenging.

3. **Magic Feature Limitations**: The AI-generated suggestions and answers don't always meet expectations, as the responses can be frequently incorrect or irrelevant, which might hinder the effectiveness of your project.

These drawbacks should be considered when making your decision on whether or not to implement AutoGen. 

TERMINATE



## Decision:

Based on the research provided by your team, I recommend choosing AutoGen for your AI Agent project. The significant advantages, such as human-in-the-loop capabilities, effective context management, scalability, and reduced development overhead, strongly support the project's requirements for a reliable and efficient AI agent system. While the high cost and issues with file imports are valid concerns, the benefits of enhanced performance and control appear to outweigh these drawbacks. The adaptability and seamless inter-agent communication further solidify its suitability for evolving project needs.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()